In [ ]:
#meep version:1.30.0
import meep as mp
import numpy as np
import math
from matplotlib import pyplot as plt

%matplotlib inline

resolution = 100  

a = 0.46  
r = 0.105  
n = 2.805  
epsilon = n**2  

theta1 = 1  
theta2 = -1  
theta1_rad = math.radians(theta1)
theta2_rad = math.radians(theta2)

sx = 16
sy = 15

nonpml_vol = mp.Volume(mp.Vector3(), size=mp.Vector3(sx-4,sy-4))

cell = mp.Vector3(sx, sy, 0)

blk = mp.Block(size=mp.Vector3(mp.inf, mp.inf, mp.inf), material=mp.Medium(epsilon=epsilon))
air1 = mp.Block(center=mp.Vector3(-10,5.4,0), size=mp.Vector3(9.75, 10, mp.inf), material=mp.Medium(epsilon=1))
air2 = mp.Block(center=mp.Vector3(-10,-5.4,0), size=mp.Vector3(9.75, 10, mp.inf), material=mp.Medium(epsilon=1))
air3 = mp.Block(center=mp.Vector3(10,-5.4,0), size=mp.Vector3(9.75, 10, mp.inf), material=mp.Medium(epsilon=1))
air4 = mp.Block(center=mp.Vector3(10,5.4,0), size=mp.Vector3(9.75, 10, mp.inf), material=mp.Medium(epsilon=1))
geometry = [blk,air1,air2,air3,air4]

def generate_triangular_lattice_points(nx=21, ny=21):
    points = []
    
    n_rows = ny - 1
    n_cols = nx - 1
    even_flag = 0  
    
    for i in range(-n_rows//2, n_rows//2 + 1):
        if even_flag == 0:
            for j in range(-n_cols//2, n_cols//2 + 1):
                x = j * a
                y = i * a * math.sqrt(3)/2
                points.append(mp.Vector3(x, y))
        else:
            for j in range(-(n_cols-1)//2, (n_cols-1)//2 + 1):
                x = j * a + a/2
                y = i * a * math.sqrt(3)/2
                points.append(mp.Vector3(x, y))
        
        even_flag = 1 - even_flag
    
    return points

lattice_points = generate_triangular_lattice_points()

for p in lattice_points:
    x_rot = p.x * math.cos(theta1_rad) - p.y * math.sin(theta1_rad)
    y_rot = p.x * math.sin(theta1_rad) + p.y * math.cos(theta1_rad)
    geometry.append(mp.Cylinder(
        radius=r,
        center=mp.Vector3(x_rot, y_rot), 
        material=mp.Medium(epsilon=1),
        height=mp.inf
    ))


for p in lattice_points:
    x_rot = p.x * math.cos(theta2_rad) - p.y * math.sin(theta2_rad)
    y_rot = p.x * math.sin(theta2_rad) + p.y * math.cos(theta2_rad)
    geometry.append(mp.Cylinder(
        radius=r,
        center=mp.Vector3(x_rot, y_rot),
        material=mp.Medium(epsilon=1),
        height=mp.inf
    ))

cylinder_centers = [
    mp.Vector3(-1.83972, 0.0321124),
    mp.Vector3(-1.83972, -0.0321124),
    mp.Vector3(-2.29965, 0.0401405),
    mp.Vector3(-2.29965, -0.0401405),
    mp.Vector3(-2.75958, 0.0481686),
    mp.Vector3(-2.75958, -0.0481686),
    mp.Vector3(-3.21951, 0.0561967),
    mp.Vector3(-3.21951, -0.0561967),
    mp.Vector3(-3.67944, 0.0642249),
    mp.Vector3(-3.67944, -0.0642249),
    mp.Vector3(-4.13937, 0.072253),
    mp.Vector3(-4.13937, -0.072253),
    mp.Vector3(1.83972, 0.0321124),
    mp.Vector3(1.83972, -0.0321124),
    mp.Vector3(2.29965, 0.0401405),
    mp.Vector3(2.29965, -0.0401405),
    mp.Vector3(2.75958, 0.0481686),
    mp.Vector3(2.75958, -0.0481686),
    mp.Vector3(3.21951, 0.0561967),
    mp.Vector3(3.21951, -0.0561967),
    mp.Vector3(3.67944, 0.0642249),
    mp.Vector3(3.67944, -0.0642249),
    mp.Vector3(4.13937, 0.072253),
    mp.Vector3(4.13937, -0.072253),
    mp.Vector3(4.5993, 0.0802811),
    mp.Vector3(4.5993, -0.0802811),
    mp.Vector3(-4.5993, 0.0802811),
    mp.Vector3(-4.5993, -0.0802811),
]

geometry.extend([
    mp.Cylinder(
        radius=r,
        center=center,
        material=mp.Medium(epsilon=epsilon),
        height=mp.inf
    )
    for center in cylinder_centers
])

def create_sector(center, radius, start_angle_deg, end_angle_deg, material, num_points=30):

    start_angle = math.radians(start_angle_deg)
    end_angle = math.radians(end_angle_deg)
    
    vertices = [center] 
    
    for i in range(num_points + 1):
        angle = start_angle + i * (end_angle - start_angle) / num_points
        point = center + mp.Vector3(radius * math.cos(angle), radius * math.sin(angle) )
        vertices.append(point)
    
    vertices.append(center)  
    
    sector = mp.Prism(vertices, height=mp.inf, material=mp.Medium(epsilon=epsilon))
    return sector

sector1 = create_sector(
    center=mp.Vector3(0, 0),
    radius=0.12,
    start_angle_deg=60,
    end_angle_deg=120,
    material=mp.Medium(epsilon=epsilon)
)
geometry.append(sector1)

sector2 = create_sector(
    center=mp.Vector3(0, 0),
    radius=0.12,
    start_angle_deg=240,
    end_angle_deg=300,
    material=mp.Medium(epsilon=epsilon)
)
geometry.append(sector2)

pml_layers = [mp.PML(2.0)]

fcen = 0.613  # pulse center frequency
df = 0.15  # pulse frequency width

sources = [
    mp.Source(
        mp.GaussianSource(fcen, fwidth=df),
        component=mp.Hz,
        center=mp.Vector3(-5.75,0,0),
        size=mp.Vector3(0,0.8),
    )
]

sim = mp.Simulation(
    cell_size=cell,
    geometry=geometry,
    boundary_layers=pml_layers,
    sources=sources,
    resolution=resolution,
)

freg = mp.FluxRegion(
    center=mp.Vector3(5.75,0,0), size=mp.Vector3(0,1,0)
)
nfreq = 1000
trans = sim.add_flux(fcen, 0.04, nfreq, freg)

dft_obj = sim.add_dft_fields(
    [mp.Hz], 
    [0.60925, 0.62], 
    where=nonpml_vol,
    yee_grid=False
)

f = plt.figure(figsize=(12, 10), dpi=1200)
sim.plot2D(ax=f.gca())
plt.tight_layout()
plt.show()

sim.run(
mp.after_sources(mp.Harminv(mp.Hz, mp.Vector3(0.1,0.1), fcen, df)),
until=1250)

freqs = mp.get_flux_freqs(trans)
psd = mp.get_fluxes(trans)

hz_data_f1 = sim.get_dft_array(dft_obj, mp.Hz, 0)  
hz_data_f2 = sim.get_dft_array(dft_obj, mp.Hz, 1)  

hz_real_f1 = np.real(hz_data_f1)
hz_real_f2 = np.real(hz_data_f2)

extent_x = (-(sx-4)/2, (sx-4)/2)
extent_y = (-(sy-4)/2, (sy-4)/2)
extent = [extent_x[0], extent_x[1], extent_y[0], extent_y[1]]

fig, axes = plt.subplots(1, 2, figsize=(32, 12), dpi=1200)

cmap = 'RdBu_r'  

ax1 = axes[0]
im1 = ax1.imshow(hz_real_f1.transpose(), 
                 cmap=cmap, 
                 interpolation='spline36',
                 origin='lower',
                 extent=extent,
                 aspect='auto')
plt.colorbar(im1, ax=ax1, label='Hz Amplitude', shrink=0.8)

ax2 = axes[1]
im2 = ax2.imshow(hz_real_f2.transpose(), 
                 cmap=cmap, 
                 interpolation='spline36',
                 origin='lower',
                 extent=extent,
                 aspect='auto')
plt.colorbar(im2, ax=ax2, label='Hz Amplitude', shrink=0.8)

plt.tight_layout()

output_filename = 'Hz.png'
plt.savefig(output_filename, dpi=1200, bbox_inches='tight', pad_inches=0.1)

plt.show()

fig = plt.figure(figsize=(11, 6), dpi=100)
ax = fig.add_subplot(111)
plt.plot(freqs, np.array(psd), "o-")
plt.grid(True)
plt.xlabel("Frequency")
plt.ylabel("Transmission")

#simulation2

cell2 = mp.Vector3(sx, sy, 0) 

blk2 = mp.Block(size=mp.Vector3(mp.inf, mp.inf, mp.inf), material=mp.Medium(epsilon=epsilon))
air5 = mp.Block(center=mp.Vector3(0,5.4,0), size=mp.Vector3(99, 10, mp.inf), material=mp.Medium(epsilon=1))
air6 = mp.Block(center=mp.Vector3(0,-5.4,0), size=mp.Vector3(99, 10, mp.inf), material=mp.Medium(epsilon=1))
geometry2 = [blk2,air5,air6]

pml_layers = [mp.PML(2.0)]

fcen = 0.613  # pulse center frequency
df = 0.15  # pulse frequency width

sources = [
    mp.Source(
        mp.GaussianSource(fcen, fwidth=df),
        component=mp.Hz,
        center=mp.Vector3(-5.75,0,0),
        size=mp.Vector3(0,0.8),
    )
]

sim2 = mp.Simulation(
    cell_size=cell2,
    geometry=geometry2,
    boundary_layers=pml_layers,
    sources=sources,
    resolution=resolution,
)

freg2 = mp.FluxRegion(
    center=mp.Vector3(5.75,0,0), size=mp.Vector3(0,1,0)
)
nfreq = 1000
trans0 = sim2.add_flux(fcen, 0.04, nfreq, freg2)

f = plt.figure(dpi=150)
sim2.plot2D(ax=f.gca())
plt.show()

sim2.run(
mp.after_sources(mp.Harminv(mp.Hz, mp.Vector3(0.1,0.1), fcen, df)),
until=1250)

psd0 = mp.get_fluxes(trans0)

fig = plt.figure(figsize=(11, 6), dpi=100)
ax = fig.add_subplot(111)
plt.plot(freqs, np.array(psd0), "o-")
plt.grid(True)
plt.xlabel("Frequency")
plt.ylabel("Transmission")

fig = plt.figure(figsize=(11, 6), dpi=100)
ax = fig.add_subplot(111)
plt.plot(freqs, np.array(psd) / np.array(psd0), "o-")
plt.grid(True)
plt.xlabel("Frequency")
plt.ylabel("Transmission")